In [31]:
 current_dir = "gdrive/My Drive/Colab Notebooks/Thesis/gold_standard"


import os
import glob
from google.colab import drive
import pandas as pd
import re
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import re
import pandas as pd

# Mount Google Drive
drive.mount('/content/gdrive')

from sklearn.linear_model import (
    LinearRegression, Ridge, Lasso, ElasticNet, Lars, LassoLars, OrthogonalMatchingPursuit,
    BayesianRidge, SGDRegressor, PassiveAggressiveRegressor, HuberRegressor,
    QuantileRegressor, LogisticRegression, RidgeCV, LassoCV, ElasticNetCV, LarsCV,
    LassoLarsCV, OrthogonalMatchingPursuitCV, ARDRegression, TweedieRegressor,
    PoissonRegressor, GammaRegressor, RANSACRegressor, TheilSenRegressor
)
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.cross_decomposition import PLSCanonical, PLSRegression, CCA
from sklearn.tree import DecisionTreeRegressor


from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import LeaveOneOut
from sklearn.metrics import mean_squared_error, r2_score
import pandas as pd
import numpy as np


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [32]:
ls

bernardo_35.txt      carl_36.txt    gold_psy.pkl    mitchell_34.txt    wayne_40.txt
bow_baselines.ipynb  dahmer_23.txt  kemper_26.txt   pos_dataframe.pkl
bundy_34.txt         gacy_27.txt    marsh_34.5.txt  ridgeway_19.txt


In [33]:
os.chdir(current_dir)

FileNotFoundError: ignored

models
features (columns)
output
name


In [59]:
copy = pd.read_pickle("pos_dataframe.pkl")
copy.head()
copy["AUX"]
for i in copy["AUX"]:
  print(i)

was was s had would was do be is m are can be be re will have could were get do is is have is will be being have
was was be is is m be
do would re might would is would is will is was can do have been had was was was was was did was was was be was was was was are have been are are are can were were are have do ve ve were was is would have been being would ve been would have been would have been would have would have have been have have been was can have ve do m are being m is is would is m is may be may be be do can would have had was are can are be are are ca being is is ve will have is
did was did ve been was did did were can was are did re is was did did do m m m were re is is did are should can am are was did ve did is do is be would is do were m m m are is s s is have m re is did re be is is was ve is is be should have been was re did have have could have have do are are is was be would are s s did did did was m are have was did s could do do re are could could ve do are could is w

## get the loocv perofrmance of a feature or feature combination

In [ ]:
features = ['ADJ', 'ADP', 'ADV', 'AUX', 'CONJ', 'CCONJ', 'DET', 'INTJ', 'NOUN',
       'NUM', 'PART', 'PRON', 'PROPN', 'PUNCT', 'SCONJ', 'SYM', 'VERB', 'X',
       'SPACE']


regression_models = [
    LinearRegression(),
    Ridge(),
    Lasso(),
    ElasticNet(),
    Lars(),
    LassoLars(),
    OrthogonalMatchingPursuit(),
    BayesianRidge(),
    SGDRegressor(),
    PassiveAggressiveRegressor(),
    HuberRegressor(),
    QuantileRegressor(),
    LogisticRegression(),
    RidgeCV(),
    LassoCV(),
    ElasticNetCV(),
    LarsCV(),
    LassoLarsCV(),
    OrthogonalMatchingPursuitCV(),
    ARDRegression(),
    TweedieRegressor(),
    PoissonRegressor(),
    GammaRegressor(),
    RANSACRegressor(),
    TheilSenRegressor(),
    SVR(),
    KNeighborsRegressor(),
    GaussianProcessRegressor(),
    PLSCanonical(),
    PLSRegression(),
    CCA(),
    DecisionTreeRegressor()
]


In [ ]:
for model, subdict in data.items():
    for submodel, details in subdict.items():
        metrics = details.get('Metrics', {})
        mse = metrics.get('Average MSE', float('inf'))
        r_squared = metrics.get('Average R^2', float('-inf'))

        if mse < 7 and r_squared > 0.8:
            filtered_data[model] = {
                submodel: details['Actual vs Predicted']
            }

print(filtered_data)

In [60]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import LeaveOneOut
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import pandas as pd
import numpy as np

def evaluate_model_with_text_column(copy, text_column, model):
    # Creating a pipeline with CountVectorizer and the specified model
    pipeline = Pipeline([
        ('vectorizer', CountVectorizer()),
        ('model', model)
    ])

    # Target variable
    y = copy['score']

    # Names of the data points (assuming the DataFrame has a 'name' column)
    names = copy['name']

    # Text data for model input
    X = copy[text_column]

    # Leave-One-Out Cross-Validation
    loo = LeaveOneOut()
    all_actual = []
    all_predicted = []
    all_names = []

    for train_index, test_index in loo.split(X):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        name_test = names.iloc[test_index].values[0]

        # Train the pipeline
        pipeline.fit(X_train, y_train)

        # Make predictions
        y_pred = pipeline.predict(X_test)
        all_predicted.extend(y_pred)
        all_actual.extend(y_test)
        all_names.append(name_test)

    # Calculate metrics
    mse = mean_squared_error(all_actual, all_predicted)
    mae = mean_absolute_error(all_actual, all_predicted)
    r2 = r2_score(all_actual, all_predicted)

    # Collect actual vs predicted values along with names
    actual_vs_predicted = [{'Name': name, 'Actual': actual, 'Predicted': pred}
                           for name, actual, pred in zip(all_names, all_actual, all_predicted)]

    # Return metrics and actual vs predicted values as a dictionary
    return {
        'Metrics': {
            'Average MSE': mse,
            'Average MAE': mae,
            'Average R^2': r2
        },
        'Actual vs Predicted': actual_vs_predicted
    }




In [61]:
scores = {}

for model in regression_models:
  scores[str(model)] = {}
  for feature in features:
    try:
      scores[str(model)][str(feature)] = evaluate_model_with_text_column(copy, feature, model)
    except:
      continue


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_stochastic_gradient.py:1548: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_stochastic_gradient.py:1548: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_stochastic_gradient.py:1548: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_stochastic_gradient.py:1548: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_stocha

In [63]:
scores

{'LinearRegression()': {'ADJ': {'Metrics': {'Average MSE': 44.50663010605284,
    'Average MAE': 5.597840381744921,
    'Average R^2': -0.10982183420118052},
   'Actual vs Predicted': [{'Name': 'wayne',
     'Actual': 40.0,
     'Predicted': 31.79095259423159},
    {'Name': 'carl', 'Actual': 36.0, 'Predicted': 32.65379674064764},
    {'Name': 'bundy', 'Actual': 34.0, 'Predicted': 28.80335300744735},
    {'Name': 'bernardo', 'Actual': 35.0, 'Predicted': 28.9779834895582},
    {'Name': 'dahmer', 'Actual': 23.0, 'Predicted': 31.887686644655524},
    {'Name': 'gacy', 'Actual': 27.0, 'Predicted': 30.62148075931582},
    {'Name': 'kemper', 'Actual': 26.0, 'Predicted': 31.82816292235495},
    {'Name': 'marsh', 'Actual': 34.5, 'Predicted': 34.177236420354376},
    {'Name': 'mitchell', 'Actual': 34.0, 'Predicted': 35.30296581265833},
    {'Name': 'ridgeway', 'Actual': 19.0, 'Predicted': 32.241429930703745}]},
  'ADP': {'Metrics': {'Average MSE': 178.8590006185794,
    'Average MAE': 11.87287120

In [66]:
filtered_data.keys()

dict_keys(['LinearRegression()', 'Ridge()', 'Lasso()', 'ElasticNet()', 'Lars()', 'LassoLars()', 'OrthogonalMatchingPursuit()', 'BayesianRidge()', 'SGDRegressor()', 'PassiveAggressiveRegressor()', 'HuberRegressor()', 'QuantileRegressor()', 'LogisticRegression()', 'RidgeCV()', 'LassoCV()', 'ElasticNetCV()', 'LarsCV()', 'LassoLarsCV()', 'OrthogonalMatchingPursuitCV()', 'ARDRegression()', 'TweedieRegressor()', 'PoissonRegressor()', 'GammaRegressor()', 'RANSACRegressor()', 'TheilSenRegressor()', 'SVR()', 'KNeighborsRegressor()', 'GaussianProcessRegressor()', 'PLSCanonical()', 'PLSRegression()', 'CCA()', 'DecisionTreeRegressor()'])

In [99]:

filtered_data = {}

for model, subdict in scores.items():
    for submodel, details in subdict.items():
        metrics = details.get('Metrics', {})
        mse = metrics.get('Average MSE', float('inf'))
        r_squared = metrics.get('Average R^2', float('-inf'))

        # Check if 'Actual vs Predicted' is not empty
        actual_vs_predicted = details.get('Actual vs Predicted', [])
        the_metrics = details.get("Metrics", [])
        if mse < 20 and r_squared > 0.5 and actual_vs_predicted:
            filtered_data[model] = {
                submodel: (f"Actual vs Predicted: {actual_vs_predicted}, Metrics: {the_metrics}")
            }




In [100]:
z = [i for i in filtered_data.items()]

In [101]:
z

[('LinearRegression()',
  {'PRON': "Actual vs Predicted: [{'Name': 'wayne', 'Actual': 40.0, 'Predicted': 34.212746991715385}, {'Name': 'carl', 'Actual': 36.0, 'Predicted': 40.919060406429956}, {'Name': 'bundy', 'Actual': 34.0, 'Predicted': 33.53746231850927}, {'Name': 'bernardo', 'Actual': 35.0, 'Predicted': 31.608200726034614}, {'Name': 'dahmer', 'Actual': 23.0, 'Predicted': 27.271591620218572}, {'Name': 'gacy', 'Actual': 27.0, 'Predicted': 31.490689672541126}, {'Name': 'kemper', 'Actual': 26.0, 'Predicted': 23.784364451194794}, {'Name': 'marsh', 'Actual': 34.5, 'Predicted': 33.14772378439099}, {'Name': 'mitchell', 'Actual': 34.0, 'Predicted': 35.159013700670094}, {'Name': 'ridgeway', 'Actual': 19.0, 'Predicted': 24.741978527224855}], Metrics: {'Average MSE': 14.887180680454966, 'Average MAE': 3.379183565523955, 'Average R^2': 0.6287717553655017}"}),
 ('Ridge()',
  {'PRON': "Actual vs Predicted: [{'Name': 'wayne', 'Actual': 40.0, 'Predicted': 34.2096634390005}, {'Name': 'carl', 'Actua

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pandas/core/indexes/base.py", line 3802, in get_loc
    return self._engine.get_loc(casted_key)
  File "pandas/_libs/index.pyx", line 138, in pandas._libs.index.IndexEngine.get_loc
  File "pandas/_libs/index.pyx", line 165, in pandas._libs.index.IndexEngine.get_loc
  File "pandas/_libs/hashtable_class_helper.pxi", line 5745, in pandas._libs.hashtable.PyObjectHashTable.get_item
  File "pandas/_libs/hashtable_class_helper.pxi", line 5753, in pandas._libs.hashtable.PyObjectHashTable.get_item
KeyError: 'target'

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-102-e2f92dc52678>", line 8, in <cell line: 8>
    y = copy['target']  # Target variable
  File "/usr/local/lib/pyth